In [1]:
import pandas as pd

<span style="color:green;font-weight:700;font-size:40px"> CLASIFICACIÓN PREVISIONAL DE PAÍSES SEGÚN SU CO2 </span>

## Tema:
El Tema u objetivo del proyecto consiste en que, dados los países del dataset, y en base a las fluctuaciones de las variables que los caracterizan, o por otra parte. Dados nuevos países que se añadan al modelo, se pueda predecir su CO2 y con ellos clasificarlos en un cluster de países previamente obtenido. Para conseguir este objetivo vamos a trabajar con machine learning no supervisado, en la parte de clusterización, y con machine learning supervisado, en la parte de predicción (regresión lineal) y clasificación (tendremos que decidir que modelo es mejor, regresión logística o k-nearest neighbors). Se establece el siguiente flujo de trabajo:

------
Clusterización de los países -> Predicción del CO2 -> Clasificación de los países
------
------

Para ello vamos a llevar a cabo tres etapas:

1. Mediante Machine Learning no supervisado vamos a obtener los clusters de nuestro dataset. En un principio partimos de la base de que queremos al menos encontrar tres clusters donde se puedan clasificar los países. Estos serían:  
    * poco contaminantes  
    * contaminantes  
    * muy contaminantes  
    
    Sin embargo, quedamos abiertos a estudiar los clusters que nos sugiera el algoritmo de machine learnning. En este caso vamos a usar K-Means y DBSCAN(elegiremos el mejor).
2. Predecir el nivel de CO2 de los países en función de una serie de variables. Para ello se plantea como base de trabajo el modelo de regresión lineal:
$$ co2 =\alpha + \beta{1}\cdot{PIB} + \beta{2}\cdot{nuclear} + \beta{3}\cdot{renovables} + \beta{4}\cdot{carbon} + beta{5}\cdot{petroleo} + \beta{6}\cdot{gas} + \beta{7}\cdot{población} + \varepsilon $$  
3. Una vez que tenga los clusters y la predicción, vamos a crear una variable target con estos mismos clusters, y con los algoritmos de machine learning supervisado:

- knn
- regresión logística  

Vamos a clasificar a los países nuevos que entren en el dataset, o a los existentes en base a variaciones de sus variables. Con esto conseguiremos una herramienta con la que poder simular escenarios futuros para dirigir la política energética y medioambiental del país, o asignar recusos a países en función de la buena gestión energética y medioambiental. Esta buena gestión vendrá dada por el cambio de cluster de uno peor a otro mejor, de tal manera que cada cluster tiene una asignación de recursos diferente a la de los demás países y, dentro del mismo cluster, una mayor asignación de recursos a aquellos países que obtengan una energía más limpia, es decir que obtengan la misma producción energética que otros países pero con menor c02.

### Datasets
El dataset está formado por datos obtenidos de la administración de energía de los Estados Unidos y contiene una colección de factores que involucran a las emisiones de CO2 para los mayores productores mundiales. Los datos recogidos van desde el año 1980 hasta el año 2019, y las variables que contiene son:
- GDP: producto interior bruto de cada país medido en billones de paridades de poder de compra (ppp) en dólares con base en el 2015.
- Population: la población del país en el año especificado, medido como millones de personas.
- Energy_production: cantidad producidad de energía, medida en quad/btu.
- Energy_consumption: cantidad de consumo de energía, medida en quad/btu.
- co2_emission: cantidad de co2 emitido, medido en miles de millones de toneladas de co2.
Per_capita_production: producción de energía per capita, medido en (quad/btu)/persona.
- Energy_intensity_by_GDP: medida de la ineficiencia energética de una economía. Está calculada como unidades de energía por unidades de GDP (1000btu/(2015$ GDP PPP)).
- balance: Indica si la economía es deficitaria o no de energía, siendo la diferencia entre la energía producida y la consumida.
- eficiency: nos dice cuanto co2 se emite por producción energética e indica que país es más limpio, medido como emisiones de c02/producción energética.
- energy_dependency: nos dice cuanta energía consume una economía por cada euro de pib generado, y por tanto nos indica la denpendencia de la economía de la energía. Se mide como consumo energético/GDP.
- use_intensity_percapita: nos dice la energía que consume cada persona en cada país,  por tanto, cuanto dependen las personas de la energía. Cuanto mayor sea este valor mayores serán las emisiones de co2 del país. Medido como consumo energético/población.
- co2_pc: nos dicrá cuanto co2 emite cada una de las personas en cada país. Se mide como emisiones de co2/población.
- Latitude: latitud del país.
- Longitude: longitud del país.
- Year: año del país.
- Country: nombre del país.
- Energy_type: tipo de energía
- Code_X: código abreviado del país.
- Continent: nombre del continente al que pertenece el país
- Geometry: geometría del país.


### Métricas para evaluar los modelos:
#### kMeans y DBSCAN:
- Elbow Method
- Silhouette Coefficient
#### LinearRegression:
- accuracy score
- mae
- mse (mean squared error)
- rmse (root mean square error)
#### LogisticRegression:
- accuracy score
- roc-auc curve
- confussion_matrix (especificity,sensibility,precision,recall)
#### k-nn:
- accuracy score
- roc-auc curve
- confussion_matrix (especificity,sensibility,precision,recall)

> Fuente de datos: [Kaggle]("https://www.kaggle.com/")

In [2]:
ruta_win = (r"C:\Users\ferna\OneDrive\Documentos\MIS_COSAS_DE_THE_BRIDGE\EDA\proyecto_co2\src\data\df_40_final_v3.csv")
df = pd.read_csv(ruta_win)
df = df.drop(columns="Unnamed: 0")
df.head()

,GDP,Population,Energy_production,Energy_consumption,CO2_emission,per_capita_production,Energy_intensity_by_GDP,balance,eficiency,energy_dependecy,use_intensity_pc,co2_pc,latitude,longitude,Year,Country,Energy_type,CODE_x,continent,geometry
0,279.389,15182.60,0.150,0.155,0.0,104.341,5.670,-0.005,0.0,0.001,0.0,0.0,6.424,-66.59,1980-01-01,Venezuela,renewables_n_other,None,South America,NaN
1,279.389,15182.60,5.000,0.812,0.0,104.341,5.670,4.188,0.0,0.003,0.0,0.0,6.424,-66.59,1980-01-01,Venezuela,petroleum_n_other_liquids,None,South America,NaN
2,279.389,15182.60,0.616,0.616,0.0,104.341,5.670,0.000,0.0,0.002,0.0,0.0,6.424,-66.59,1980-01-01,Venezuela,natural_gas,None,South America,NaN
3,279.389,15182.60,0.001,0.002,0.0,104.341,5.670,-0.000,0.0,0.000,0.0,0.0,6.424,-66.59,1980-01-01,Venezuela,coal,None,South America,NaN
4,278.376,15597.88,0.156,0.161,0.0,110.257,6.178,-0.005,0.0,0.001,0.0,0.0,6.424,-66.59,1981-01-01,Venezuela,renewables_n_other,None,South America,NaN


In [8]:
none = df[df.CODE_x == "None"]
none.Country.unique()

array(['Venezuela', 'Iran', 'Russia', 'Former Czechoslovakia',
       'Former Yugoslavia'], dtype=object)

El iso code obtenido en [International Iso Country Codes](https://www.countrycode.org/venezuela) son:
- Venezuela: VEN
- Rusia: RUS
- Iran: IRN
- República Checa: CZE
- yugoslavia no existe ahora es serbia: SRB